# io power - Samna only - v0.39.9

- Use PDM_CLK and PDM_DATA driven by FPGA to XA3:
- PDM_CLK and PDM_DATA toggle in the normal way
- only PDM_CLK toggles
- only PDM_DATA toggles
- PDM_CLK and PDM_DATA stays at all level combinations:
    - PDM_CLK=0, PDM_DATA=0
    - PDM_CLK=0, PDM_DATA=1
    - PDM_CLK=1, PDM_DATA=0
    - PDM_CLK=1, PDM_DATA=1

### Initialization and connection to the board

In [1]:
import samna
print(f'Samna version {samna.__version__}')
import numpy as np

# Open the device and connect it to the power monitor.
xylo_node = samna.device.open_device("XyloAudio3TestBoard")

power_monitor = xylo_node.get_power_monitor()
sink_pm = samna.graph.sink_from(power_monitor.get_source_node())
stopwatch = xylo_node.get_stop_watch()
io = xylo_node.get_io_module()

xylo_model = xylo_node.get_model()
source = samna.graph.source_to(xylo_model.get_sink_node())

# We are only interested in Readout events, so we make a filter graph to filter only these events.
# Important note: `graph` needs to be kept alive for the filter to work.
graph = samna.graph.EventFilterGraph()
_, etf, readout_sink = graph.sequential([xylo_model.get_source_node(), "XyloAudio3OutputEventTypeFilter", samna.graph.JitSink()])
etf.set_desired_type('xyloAudio3::event::Readout')


## With the network, clock frequency can not go low as 6.25
clock_frequencies = [12.5, 25, 50]
ts = 1e-3

power_measurements_625 = []
power_measurements_125 = []
power_measurements_25 = []
power_measurements_50 = []

Samna version 0.39.9.0


In [2]:
disable_main_clock = False

## Create a configuration to map a network on the chip

small == SynNet([24, 24, 24]) 

trained == LIFTorch([63,63,63])

In [3]:
map_network = True

In [4]:
from rockpool.nn.networks import SynNet
from rockpool.devices.xylo.syns65302 import config_from_specification, mapper

xylo_config = []

if map_network:
    from rockpool.nn.modules import LIFTorch
    import warnings
    warnings.filterwarnings("ignore")
    
    ckpt = '../../../docs/devices/xylo-a3/model_sample/to_deploy_inXylo.json'
    
    # - Loading trained model architecture parameters
    arch_params = {'n_classes': 1,
    'n_channels': 16,
    'size_hidden_layers':[63, 63, 63],
    'time_constants_per_layer':[3,7,7],
    'tau_syn_base': 0.02,
    'tau_mem': 0.02,
    'tau_syn_out': 0.02,
    'neuron_model': LIFTorch,
    'dt': 0.00994,
    'output': 'vmem'}
    
    # - Instantiating the model backbone and loading trained checkpoint
    syn_model = SynNet(** arch_params)
    syn_model.load(ckpt)
    
    import rockpool.transform.quantize_methods as q
    
    # getting the model specifications using the mapper function
    spec = mapper(syn_model.as_graph(), weight_dtype='float', threshold_dtype='float', dash_dtype='float')
    # quantizing the model
    spec.update(q.channel_quantize(**spec))
    
    xylo_config, is_valid, msg = config_from_specification(**spec)
    
else:
    # network to be configured in each step
    xylo_config = samna.xyloAudio3.configuration.XyloConfiguration()

print(xylo_config)

WARNING    /home/vleite/SynSense/rockpool/rockpool/nn/networks/__init__.py:15: UserWarning: This module needs to be ported to the v2 API.
  warnings.warn(f"{err}")
 [py.warnings]
WARNING    /home/vleite/SynSense/rockpool/rockpool/nn/networks/__init__.py:20: UserWarning: This module needs to be ported to the v2 API.
  warnings.warn(f"{err}")
 [py.warnings]


xyloAudio3::configuration::XyloConfiguration(operation_mode=OperationMode.AcceleratedTime, input_source=InputSource.SpikeEvents, synapse2_enable=0, bias_enable=1, time_resolution_wrap=0, output_counter_wrap=0, enable_hibernation_mode=0, hibernation_mode_time_resolution_wrap=0, input=xyloAudio3::configuration::InputConfig(weight_bit_shift=0, weights={ -61 -35 91 39 -68 -43 127 22 33 6 -12 -3 39 -41 76 22 22 29 -20 -127 38 -20 22 0 -59 16 -43 -44 -16 21 -14 -59 24 90 5 -26 65 -44 49 -43 54 46 68 -61 -105 48 -5 2 -22 -17 12 127 6 44 -38 33 -127 -1 108 -100 127 15 127 5 127 49 20 -32 -127 79 26 31 -81 -17 0 111 127 -127 -34 -111 -45 -52 -46 -17 -74 -7 -45 73 -85 -26 -32 -63 -27 -46 -32 -127 38 -15 -10 9 -10 -18 93 -2 -43 40 -14 104 116 26 18 34 -127 -127 87 23 21 100 66 -84 -50 -65 24 24 -75 -52 -74 -64 109 15 38 -66 90 1 8 -51 -49 123 -90 -8 -64 -79 122 -47 127 62 12 82 -17 30 -9 -127 2 -15 -125 16 -69 -63 15 47 41 -31 6 -14 -127 -87 -42 -42 -47 73 60 127 59 -63 -32 -27 -37 22 0 3 -22 -93

## Create evolve function with pure Samna to activate chip processing

In [5]:
def get_current_timestep():
    """Utility function to obtain the current timestep.

    The `Readout` event always contains the current timestep so we could simply save it.
    But in case you forgot what timestep was last processed, you can obtain it as follows.
    """
    source.write([samna.xyloAudio3.event.TriggerReadout()])
    evts = readout_sink.get_n_events(1, timeout=3000)
    assert(len(evts) == 1)
    return evts[0].timestep


def evolve(input):
    """Continue to evolve the model with the given input.

    Args:
        input (list[list[int]]): Per timestep a list of integers to specify the number of spikes to send to that `neuron_id`.
                                Max number of spikes per timestep for a neuron is 15. All lists must have the same length.

    Returns:
        readouts (list[Readout]): The `Readout` events for the given timesteps.
    """
    timestep_count = len(input)
    if not timestep_count:
        return []

    start_timestep = get_current_timestep() + 1
    final_timestep = start_timestep + timestep_count - 1
    # print("** To be processed timesteps: ", [timestep for timestep in range(start_timestep, final_timestep + 1)])

    input_events_list = []
    for i, spike_counts in enumerate(input):
        timestep = start_timestep + i
        for neuron_id, count in enumerate(spike_counts):
            spikes = [samna.xyloAudio3.event.Spike(neuron_id=neuron_id, timestep=timestep)] * count
            input_events_list.extend(spikes)

    input_events_list.append(samna.xyloAudio3.event.TriggerProcessing(target_timestep = final_timestep + 1))
    source.write(input_events_list)

    events = readout_sink.get_n_events(timestep_count, timeout=3000)
    assert(len(events) == timestep_count)

    # for idx, ev in enumerate(events):
    #         print(f"{idx}: {ev}")

    return events

### Encode a PDM input chirp

In [12]:
# - Encode a PDM input chirp
from rockpool.devices.xylo.syns65302 import AFESimPDM
# from rockpool.devices.xylo.syns65302.afe import PDM_SAMPLING_RATE

import numpy as np 
from scipy.signal import chirp

PDM_SAMPLING_RATE = 1 * 1000000

pdm_sr = PDM_SAMPLING_RATE
print(pdm_sr)
net_dt = 1024e-6
audio_sr = PDM_SAMPLING_RATE # 48e3

afesim_pdm = AFESimPDM.from_specification(
    select_filters=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15),
    spike_gen_mode="divisive_norm",
    # spike_gen_mode="threshold",
    # fixed_threshold_vec = [2**25 for i in range(16)],
    rate_scale_factor=63,
    low_pass_averaging_window=84e-3,
    dn_EPS=1,
    fixed_threshold_vec=None,
    dt=net_dt,
)

dur = 200e-3

T = int(audio_sr * dur)
f_start = 1000
f_end = 20e3
times = np.arange(0, T) / audio_sr
signal = chirp(times, f_start, T/audio_sr, f_end, method='linear', phi = 90)
# signal = np.sin(2 * np.pi * f_start * times)

__scale = 0.95
out_pdm, state_pdm, rec_pdm = afesim_pdm((signal * __scale, audio_sr))

input_pdm = (rec_pdm['0_PDMADC']['0_MicrophonePDM_output'] + 1) / 2

print(input_pdm)

PDM_SAMPLES_PER_DT = int(PDM_SAMPLING_RATE * net_dt)
num_dt = np.size(input_pdm) // PDM_SAMPLES_PER_DT

# - Bin samples into `dt`-length windows and trim
input_raster = np.reshape(
    input_pdm[: int(num_dt * PDM_SAMPLES_PER_DT)], [-1, PDM_SAMPLES_PER_DT]
)

print(len(input_raster))


1000000
[0. 1. 1. ... 1. 0. 1.]
305


In [7]:
try:
    from tqdm.autonotebook import tqdm
except:
    tqdm = lambda x: x

### Configure PDM_CLK at 1 MHz and Main clock

In [8]:
def configure_clocks():

    if (disable_main_clock):
        # Defines PDM clock rate
        config = samna.xyloAudio3.XyloAudio3TestBoardDefaultConfig()
        config.pdm_clock_frequency = 1000000
        xylo_node.reset_board_soft(config)

        # Disable main clock
        io.write_config(0x0008, 0)

    else:
        # Defines PDM clock rate
        config = samna.xyloAudio3.XyloAudio3TestBoardDefaultConfig()
        config.main_clock_frequency = 6250000
        config.pdm_clock_frequency = 1000000
        xylo_node.reset_board_soft(config)



### Use PDM_CLK and PDM_DATA driven by FPGA:
- Network mapped and main clock disabled
  
- PDM_CLK and PDM_DATA toggle in the normal way:
    * PDM_CLK toggles at ~ 1 MHz
    * PDM_DATA toggles with random data, not constant zero or one, but no specific pattern requirement either

In [9]:
import time

# Feed PDM_CLK externally
xylo_config.digital_frontend.pdm_preprocessing.clock_direction = 1

configure_clocks()

xylo_node.get_model().apply_configuration(xylo_config)

# PDM data
xylo_config.input_source = samna.xyloAudio3.InputSource.PdmEvents

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)
power_events = []
for input_sample in tqdm(input_raster):
    # - Send PDM events for this dt
    pdm_events = [
        samna.xyloAudio3.event.AfeSample(data=int(i)) for i in input_sample
    ]

    print(pdm_events)
    break
    
    source.write(pdm_events)
    time.sleep(0.5)
    source.write([samna.xyloAudio3.event.TriggerProcessing()])

    # Auto power measurement streams PowerMeasurement events continuously.
    power_events += sink_pm.get_events()
    # power_events.append(events)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in power_events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')


  0%|          | 0/305 [00:00<?, ?it/s]

[xyloAudio3::event::AfeSample(data=0), xyloAudio3::event::AfeSample(data=1), xyloAudio3::event::AfeSample(data=1), xyloAudio3::event::AfeSample(data=0), xyloAudio3::event::AfeSample(data=1), xyloAudio3::event::AfeSample(data=0), xyloAudio3::event::AfeSample(data=0), xyloAudio3::event::AfeSample(data=1), xyloAudio3::event::AfeSample(data=1), xyloAudio3::event::AfeSample(data=0), xyloAudio3::event::AfeSample(data=0), xyloAudio3::event::AfeSample(data=1), xyloAudio3::event::AfeSample(data=0), xyloAudio3::event::AfeSample(data=1), xyloAudio3::event::AfeSample(data=1), xyloAudio3::event::AfeSample(data=0), xyloAudio3::event::AfeSample(data=0), xyloAudio3::event::AfeSample(data=1), xyloAudio3::event::AfeSample(data=1), xyloAudio3::event::AfeSample(data=0), xyloAudio3::event::AfeSample(data=0), xyloAudio3::event::AfeSample(data=1), xyloAudio3::event::AfeSample(data=1), xyloAudio3::event::AfeSample(data=0), xyloAudio3::event::AfeSample(data=0), xyloAudio3::event::AfeSample(data=1), xyloAudio3:

ZeroDivisionError: division by zero

### Use PDM_CLK and PDM_DATA driven by FPGA:
- Network mapped and main clock disabled
  
- PDM_CLK only toggle:
    * PDM_CLK toggles at ~ 1 MHz
    * PDM_DATA constant zero

In [17]:
import time

# Feed PDM_CLK externally
xylo_config.digital_frontend.pdm_preprocessing.clock_direction = 1

configure_clocks()

xylo_node.get_model().apply_configuration(xylo_config)

# PDM data
xylo_config.input_source = samna.xyloAudio3.InputSource.PdmEvents

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)

# Send PDM data
power_events = []
for input_sample in tqdm(input_raster):
    # - Send all zeros with same size samples as the chirp
    pdm_events = [
        samna.xyloAudio3.event.AfeSample(data=int(0)) for i in input_sample
    ]
    source.write(pdm_events)
    time.sleep(0.5)
    source.write([samna.xyloAudio3.event.TriggerProcessing()])

    # Auto power measurement streams PowerMeasurement events continuously.
    power_events += sink_pm.get_events()
    # power_events.append(events)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in power_events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')



  0%|          | 0/305 [00:00<?, ?it/s]

io:	50 uW	AFE core:	13 uW	DFE+SNN core:	633 uW


### Use PDM_CLK and PDM_DATA driven by XyloAudio 3:
- Network mapped and main clock disabled
  
- PDM_CLK and PDM_DATA toggle in the normal way:
    * PDM_CLK toggles at ~ 1 MHz
    * PDM_DATA toggles with random data, not constant zero or one, but no specific pattern requirement either

In [9]:
import time

# Feed PDM_CLK internally
xylo_config.digital_frontend.pdm_preprocessing.clock_direction = 0

configure_clocks()

xylo_node.get_model().apply_configuration(xylo_config)

# PDM data
xylo_config.input_source = samna.xyloAudio3.InputSource.PdmEvents

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)
power_events = []
for input_sample in tqdm(input_raster):
    # - Send PDM events for this dt
    pdm_events = [
        samna.xyloAudio3.event.AfeSample(data=int(i)) for i in input_sample
    ]
    source.write(pdm_events)
    time.sleep(0.5)
    source.write([samna.xyloAudio3.event.TriggerProcessing()])

    # Auto power measurement streams PowerMeasurement events continuously.
    power_events += sink_pm.get_events()
    # power_events.append(events)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in power_events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')



  0%|          | 0/305 [00:00<?, ?it/s]

io:	50 uW	AFE core:	14 uW	DFE+SNN core:	633 uW


### Use PDM_CLK and PDM_DATA driven by XyloAudio 3:
- Network mapped and main clock disabled
  
- only PDM_CLK toggle:
    * PDM_CLK toggles at ~ 1 MHz
    * PDM_DATA constant zero

In [9]:
import time

# Feed PDM_CLK internally
xylo_config.digital_frontend.pdm_preprocessing.clock_direction = 0

configure_clocks()

xylo_node.get_model().apply_configuration(xylo_config)

# PDM data
xylo_config.input_source = samna.xyloAudio3.InputSource.PdmEvents

# Start the stopwatch to enable time-stamped power sampling.
stopwatch.start()

# Start sampling power on all channels at a rate of 100 Hz.
power_monitor.start_auto_power_measurement(100)
power_events = []
for input_sample in tqdm(input_raster):
    # - Send PDM events for this dt
    pdm_events = [
        samna.xyloAudio3.event.AfeSample(data=int(0)) for i in input_sample
    ]
    source.write(pdm_events)
    time.sleep(0.5)
    source.write([samna.xyloAudio3.event.TriggerProcessing()])

    # Auto power measurement streams PowerMeasurement events continuously.
    power_events += sink_pm.get_events()
    # power_events.append(events)

# Stop the automatic power measurement process.
power_monitor.stop_auto_power_measurement()

# Calculate the average power consumption for each channel over 3 seconds.
counts = [0, 0, 0]
sums = [0, 0, 0]
for e in power_events:
    assert isinstance(e, samna.unifirm.modules.events.PowerMeasurement)
    sums[e.channel] += e.value
    counts[e.channel] += 1

# Compute the averages and convert to milliwatts (W -> mW).
avgs = [sum/count * 1000 for sum, count in zip(sums, counts)]

print(f'io:\t{np.ceil(avgs[0] * 1000):.0f} uW\tAFE core:\t{np.ceil(avgs[1] * 1000):.0f} uW\tDFE+SNN core:\t{np.ceil(avgs[2] * 1000):.0f} uW')



  0%|          | 0/305 [00:00<?, ?it/s]

io:	50 uW	AFE core:	14 uW	DFE+SNN core:	633 uW
